# Automated Substation Creation and Feeder Insertion in Native CIM17

CIM-Builder is a new python library developed under the Grid Atlas and MAPLE LEAF projects for creating CIM models "from scratch" with no pre-existing model files. This is a significantly different capability than all other tooling, which requires a source file from which to start, such as OpenDSS, PSSE, or GIS data. The library currently inlcudes three main functionalities:

1) automatic creation of new node-breaker substations in CIM based on interactive API calls

2) automatic insertion of existing distribution feeders in CIM into new node-breaker substations

3) automatic insertion of new aggregate feeder data in existing CIM transmission models (not demonstrated in this report). //

The library contains a set of classes for each type of common substation, which each contain two methods. The first is `add_feeder`, which automatically adds the specified distribution feeder to the substation and creates the chain of switching equipment between the sourcebus of the feeder and the correct breaker or airgap switch in the substation. The second is `add_branch`, which enables automated addition of transmission branches, transformers, and shunt equipment to the substation bus.

The classes supported within the first version of CIM-Builder are

* `SingleBusSubstation`
* `MainAndTransferSubstation`
* `RingBusSubstation`
* `DoubleBusSingleBreakerSubstation`
* `BreakerAndHalfSubstation`

When instantiated, these classes create a new CIMantic Graphs `DistributedArea` or `NodeBreakerModel` graph model with all CIM objects associated with the default bus configuration for that substation. Distribution feeders can then be added to the substation by instantiating each feeder as a new CIMantic Graphs `FeederModel`. The python library then builds the set of CIM associations to map the feeder to the substation and create all CIM objects for the breaker, airgap switches, and junctions.



----

### Import CIMantic Graphs Library and CIM profile

The CIMantic Graphs library can be installed using `pip install cim-graph` and imported into any script:

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
import cimgraph.utils as utils

CIMantic Graphs can support multiple CIM profiles, so it is necessary to specify which one to use. This demonstration will use the extensions from CIMHub feature/SETO, which is derived from CIM17V38.

In [2]:
# Specify CIM Profile
import importlib
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

To create a new model "from scratch", no filename or databased is specified. This connection object contains all information regarding the CIM version, serilaization format, etc.

In [3]:
# Create an empty connection for new file
params = ConnectionParameters(filename=None,
                              cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

For this demonstration, the IEEE 13 and IEEE 13 assets feeders will be used to populate each substation. It is possible to load these files from a database, or open them directly from the XML using CIMantic Graphs. Since end-users of the Grid Atlas may not have a database, the feeders will be loaded from the standalone files:

In [4]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml',
                               cim_profile=cim_profile, iec61970_301=8)
ieee13_connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=ieee13_connection, container=ieee13_feeder,
                              distributed=False)


In [5]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', 
                              cim_profile=cim_profile, iec61970_301=8)
assets13_connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=assets13_connection, container=assets13_feeder,
                                distributed=False)


For this demonstration, the Blazegraph database used by GridAPPS-D will be used for roundtrip testing. During these tests, each of the substation models developed for Grid Atlas will be added to the database. Connectivity and total load within the test feeders will be queried through a simple test.

To connect to the database, we will establish two connections. The first will be used for database bulk upload using the CIM-Loader library previously developed for Grid Atlas. The second will be used for automated database queries using CIMantic Graphs.

In [6]:
# Connection to Blazegraph Database for CIM-Loader
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                               cim_profile=cim_profile, iec61970_301=8)
bg_loader = BlazegraphLoader(params)

In [7]:
# Connection to Blazegraph Database for CIMantic Graphs
from cimgraph.databases import BlazegraphConnection as BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql",
                               cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)

### Create New Single Bus Substation

To create a new Single Bus substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [8]:
from cimbuilder.substation_builder import SingleBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [9]:
SubBuilder = SingleBusSubstation(connection=connection,
                                 name="single_bus_sub", 
                                 base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches.The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [10]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [11]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "cb8c7601-0e43-478c-b206-be1137f45707": {
        "mRID": "cb8c7601-0e43-478c-b206-be1137f45707",
        "name": "single_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9E

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [12]:
utils.write_xml(SubBuilder.network, '../test_output/single_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [13]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=18ms, elapsed=18ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=20ms, commitTime=1709076712684, mutationCount=6789</p\n></html\n>'

In [14]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/single_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="173" milliseconds="18"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [15]:
network = NodeBreakerModel(container=substation, connection=blazegraph)

In [16]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "cb8c7601-0e43-478c-b206-be1137f45707": {
        "mRID": "cb8c7601-0e43-478c-b206-be1137f45707",
        "name": "single_bus_sub",
        "ConnectivityNodes": "['c6116ccc-4f54-477a-81f4-08af12caee0a', '09a344f1-ed78-403c-85c6-6b882c7dba66', '26dc1181-0bbd-47f6-ac32-901f12bde92c', '2a8098d4-dcdc-4d68-9507-d902e493882b', '6361f281-a755-4fb6-8638-b02fa46647a6']",
        "Equipments": "['941292aa-5a87-4daa-a812-77a31b83c90f', 'aa393741-2984-4dd3-a57d-8054dc5070bc', 'c8dea6bd-cf7e-4389-b8d7-a5e5931a4c94', '3dd99962-b842-4658-917a-b41c950f471b', '641c7b84-4d0b-463f-94ca-0834c5f88173', '72e9cf93-9c12-49b5-85e1-6ea1a5097662', '7d45b85d-8f46-407a-a11b-576d6c3d3d33']",
        "NormalEnergizedFeeder": "['49AD8E07-3BF9-A4E2-CB8F-C3722F837B62', '5B816B93-7A5F-B64C-8460-47C17D6E4B0F']"
    }
}


In [17]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [18]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [19]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/single_bus_and_feeders.xml')

### Create New Sectionalized-Bus Substation

To create a new Sectionalized-Bus substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [20]:
from cimbuilder.substation_builder import SectionalizedBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections is specified by `total_sections` with a default value of two bus sections.

In [21]:
SubBuilder = SectionalizedBusSubstation(connection=connection,
                                        name="sectionalized_sub", 
                                        base_voltage = 115000,
                                        total_sections = 2)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `section_number` is used to identify the bus section to which the feeder should be connected. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [22]:
SubBuilder.new_feeder(section_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(section_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [23]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "58449cb8-8a5f-4cad-abe7-62a6cc2abe01": {
        "mRID": "58449cb8-8a5f-4cad-abe7-62a6cc2abe01",
        "name": "sectionalized_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [24]:
utils.write_xml(SubBuilder.network, '../test_output/sectionalized.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [25]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=17ms, elapsed=17ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=20ms, commitTime=1709076717039, mutationCount=6461</p\n></html\n>'

In [26]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/sectionalized.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="19"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="23"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="252" milliseconds="20"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [27]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [28]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "58449cb8-8a5f-4cad-abe7-62a6cc2abe01": {
        "mRID": "58449cb8-8a5f-4cad-abe7-62a6cc2abe01",
        "name": "sectionalized_sub",
        "ConnectivityNodes": "['890364d1-1a9f-43eb-b649-b6ce9d045337', 'adbce424-2d9c-459b-bf9a-a5cb7069a71f', 'aff45da7-3c2e-4d84-89a3-35560e707422', 'b7b5f302-5872-4c64-be47-cc0b2eae6c76', 'c81cb174-de0f-401d-ac91-a02aadd9c7b6', 'd32b6891-a2c9-408e-8eee-6327771512dc', 'd7d243eb-8973-48a0-b434-c56a35eabef2', '0dee55b9-a354-47fb-923f-4fe93b8df821']",
        "Equipments": "['88c82476-ca31-473f-87a4-ded28ecceae2', '89552179-0bc7-4fb9-b15b-5fb3f9b5ba37', '9aaffb28-0f72-40a0-aa73-38e3033cbfdd', 'cc609993-2d83-4faf-a959-9757560cff0e', 'd9fa753f-fae1-4d10-9a9e-49a7b0381df6', 'e3978b8c-a59f-4b05-b442-48bb177515a1', 'f0c9ab09-147e-47f3-b27e-4b52e0f307fc', '1195372d-175c-4ef1-ad1c-1c1c15979907', '4d106def-cca7-4125-897d-818f0002d50d', '64fcb403-a499-45ab-9aa2-532392d7cde9', '7eef8843-1fdb-447b-b7d1-e10bb51eb8ce']",
        "NormalEnergizedFeeder": "['49AD

In [29]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [30]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [31]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/sectionalized_and_feeders.xml')

### Create New Main-and-Transfer Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [32]:
from cimbuilder.substation_builder import MainAndTransferSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [33]:
SubBuilder = MainAndTransferSubstation(connection=connection,
                                       name="main_transfer_sub", 
                                       base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [34]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [35]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "5ce025e2-6352-48db-8cdd-dd6c5dc41e65": {
        "mRID": "5ce025e2-6352-48db-8cdd-dd6c5dc41e65",
        "name": "main_transfer_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [36]:
utils.write_xml(SubBuilder.network, '../test_output/main_and_transfer.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [37]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=20ms, elapsed=20ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=23ms, commitTime=1709076721659, mutationCount=6540</p\n></html\n>'

In [38]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/main_and_transfer.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="25"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="25"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="290" milliseconds="20"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [39]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [40]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "5ce025e2-6352-48db-8cdd-dd6c5dc41e65": {
        "mRID": "5ce025e2-6352-48db-8cdd-dd6c5dc41e65",
        "name": "main_transfer_sub",
        "ConnectivityNodes": "['8b1ee5c2-2cbe-48e4-a22b-fee2fcf644dd', '9f883dce-8992-465a-9af0-0e709ea84251', 'a72bbb03-1639-40b8-b813-fd0dbd84f0f5', 'd64ab296-10d0-4d4b-a96a-92a3aa9dd1c5', 'd9dc9438-c393-4d06-ba54-22635389b59d', '06812483-5178-46c3-aaee-0c5a7e733398', '324277eb-aec0-4aa8-9a60-ca8872cede60', '32dd0ede-4e74-4317-b40b-30e6f2274871']",
        "Equipments": "['8511633b-69a1-4c86-8951-3561aaf79626', '9d017259-5665-48d9-814a-92fb61318caf', 'ae499193-cd04-4340-89b9-fba0b590934f', 'bea576c9-04ef-4aaf-8cf2-492a56377b77', 'fddc00fc-9545-466d-b1e3-20805efc0673', '0f06e820-c294-4a4c-9358-f2714fbe8dc6', '10d9a098-6e91-45f4-84ae-d5c349713b4e', '1ea5b1a4-248f-4725-8154-24f00de6992b', '2ebb6495-54a3-4d7d-a6a9-523518320070', '40932d50-444a-4df0-b1c8-cfefef50dd68', '4973c48f-447f-4642-8928-02638220a285', '620f2805-1e52-4b68-a042-d6b50be2d184', '7

In [41]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [42]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [43]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/main_transfer_and_feeders.xml')

### Create Double Bus Single Breaker Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [44]:
from cimbuilder.substation_builder import DoubleBusSingleBreakerSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation.

In [45]:
SubBuilder = DoubleBusSingleBreakerSubstation(connection=connection,
                                              name="double_bus_sub", 
                                              base_voltage = 115000)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [46]:
SubBuilder.new_feeder(series_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [47]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "6169a4c6-53a9-4167-960d-ee0e6f0b197d": {
        "mRID": "6169a4c6-53a9-4167-960d-ee0e6f0b197d",
        "name": "double_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9E

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [48]:
utils.write_xml(SubBuilder.network, '../test_output/double_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [49]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=17ms, elapsed=17ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=21ms, commitTime=1709076726115, mutationCount=6578</p\n></html\n>'

In [50]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/double_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="19"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="244" milliseconds="22"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [51]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [52]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "6169a4c6-53a9-4167-960d-ee0e6f0b197d": {
        "mRID": "6169a4c6-53a9-4167-960d-ee0e6f0b197d",
        "name": "double_bus_sub",
        "ConnectivityNodes": "['890c129c-c8b8-43a6-a1aa-4ed391067c0a', '8f0f5dd7-02fd-4f99-b6d3-b885ebd5663e', 'e7e5241b-5a7a-4225-8736-ff99cffd118e', '3aaf3cf9-3768-4b42-98bf-643415e12880', '7161e04b-896f-4a35-86d4-d68a953be1fa', '71a9be64-9c7d-42c8-ae6a-4362b78d7f52']",
        "Equipments": "['9cb32476-ae7f-4d5b-9718-a2bb7dd8b2c7', 'b3d458df-fa29-4848-b605-cf2bba880e3f', 'c2b941fe-ea27-440a-9ab1-12670061762f', 'cf9281b3-1dc8-4a60-bd15-d06c6ebc62a4', '024092b4-2e8e-4b42-8380-0cf7a1618cfb', '1749a651-bfba-4fd8-af64-14dc469f436b', '194ab054-b558-46a8-bd18-e3e3823b69f4', '2967826e-6f23-4bc3-8255-65658dd9fda1', '5aeaf411-35f9-4926-a56b-a08d57eb9754', '6fc21c8b-27d7-4716-a66c-03e19b795e4b', '73dc6f45-a313-4b78-be3f-ab04bc80dcec']",
        "NormalEnergizedFeeder": "['49AD8E07-3BF9-A4E2-CB8F-C3722F837B62', '5B816B93-7A5F-B64C-8460-47C17D6E4B0F']"
    }
}

In [53]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [54]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [55]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/double_bus_and_feeders.xml')

### Create Ring Bus Substation

To create a new Main and Transfer substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [56]:
from cimbuilder.substation_builder import RingBusSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of bus sections within the ring is denoted by `total_sections`.

In [57]:
SubBuilder = RingBusSubstation(connection=connection,
                               name = "ring_bus_sub", 
                               base_voltage = 115000,
                               total_sections = 4)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [58]:
SubBuilder.new_feeder(bus_number = 1, feeder=ieee13_feeder,
                       feeder_network=ieee13_network)
SubBuilder.new_feeder(bus_number = 2, feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [59]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "8a4a3b73-75ae-4119-a685-b998613d469b": {
        "mRID": "8a4a3b73-75ae-4119-a685-b998613d469b",
        "name": "ring_bus_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED0

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [60]:
utils.write_xml(SubBuilder.network, '../test_output/ring_bus.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [61]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=18ms, elapsed=18ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=23ms, commitTime=1709076730332, mutationCount=6532</p\n></html\n>'

In [62]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/ring_bus.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="19"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="22"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="383" milliseconds="22"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [63]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [64]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "8a4a3b73-75ae-4119-a685-b998613d469b": {
        "mRID": "8a4a3b73-75ae-4119-a685-b998613d469b",
        "name": "ring_bus_sub",
        "ConnectivityNodes": "['c2dfed47-2edb-477a-a990-cfab3a180212', 'e92f265e-8fbc-4ced-8075-d214ed222d44', '0f0f6aa3-2f3e-402e-b37e-b3f38cd3fbde', '16772a8c-34d9-4ade-8f27-90b54bb0a267', '17136127-d02b-4f0b-ad8b-00f4ff75b398', '3f9f32a7-6bd6-40c1-890e-29b64414d25e', '555d19ef-77b0-4cbf-890b-4180c5415c77', '60f67759-b1c4-451a-9e7b-66544439ba6e', '658382a5-291f-4620-b5ed-11d950b3cfd9', '739bbf1e-3b3a-4261-979f-f3983eadfa77', '77df54fc-e1fa-479c-95d0-591097ec22ba', '7de2624a-c38d-498f-ae84-e591dded91f4']",
        "Equipments": "['8494989e-7526-4b62-8714-63c6a48cc8d6', '92c1f834-99ad-4700-b687-de44c2429459', '944468ab-b5da-4828-908d-b439bb517525', '99e54061-f150-4549-acde-8be855b0d24d', '9e012dd8-9f7d-4e58-abfa-818dda41ad4b', 'b623e89e-b403-455e-90b7-fe3d2ed2db34', 'c5317ad7-3e9c-4252-b0f8-3ba475384621', 'cc7065d1-8e3f-4970-a919-22159c5e4552', 'ea8fff

In [65]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [66]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [67]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/ring_bus_and_feeders.xml')

### Create Breaker and a Half Substation

To create a new Breaker-and-a-Half substation, we import the associated class from the `substation_builder` module in CIM-Builder:

In [68]:
from cimbuilder.substation_builder import BreakerAndHalfSubstation

To create a new substation, the class is instantiated with the name of the substation and base voltage. If the substation is not being added to an existing network, a new `DistributedArea` will be created to contain the substation. The total number of breaker sets in the substation is denoted by `total_bus_ties`.

In [69]:
SubBuilder = BreakerAndHalfSubstation(connection=connection,
                               name = "breaker_half_sub", 
                               base_voltage = 115000,
                               total_bus_ties=2)
substation = SubBuilder.substation

To add a feeder to the new substation, the `.new_feeder()` method is called. The `series_number` is used for automatic naming of the breaker and airgap switches. The arguments `feeder` and `feeder_network` are used for the CIM feeder object and the CIMantic Graphs network model.

The function call below will automatically insert the IEEE 13 and IEEE 13 Assets feeders into the new substation:

In [70]:
SubBuilder.new_feeder(branch_number=1, tie_number=1,
                      feeder=ieee13_feeder,
                      feeder_network=ieee13_network)
SubBuilder.new_feeder(branch_number = 2, tie_number=1,
                      feeder=assets13_feeder,
                      feeder_network=assets13_network)

To check whether the feeders have been added successfully, it is possible to use the CIMantic Graphs `.pprint(cim.ClassName)` method to print all instances of Substation and Feeder objects.

In [71]:
# Print substation data using names
SubBuilder.network.pprint(cim.Substation, use_names = True)
# Print feeder data
SubBuilder.network.pprint(cim.Feeder)

{
    "ed5fc824-37f7-4cd0-9308-0f5a1aecbcf6": {
        "mRID": "ed5fc824-37f7-4cd0-9308-0f5a1aecbcf6",
        "name": "breaker_half_sub",
        "NormalEnergizedFeeder": "['ieee13nodeckt', 'ieee13nodecktassets']"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F

The CIMantic Graphs `utils` package can be used to save the substation to an XML file for further use. 

In [72]:
utils.write_xml(SubBuilder.network, '../test_output/breaker_half.xml')

The final verification that the substation has been created and can be successfully used with the network models Round-Trip Test: 
1) Delete all old models from the database
2) Load the new substation and feeder CIM XML files
3) Obtain all data for feeders contained by the subustation

In [73]:
# Delete all CIM triples from database
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=22ms, elapsed=22ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=27ms, commitTime=1709076734889, mutationCount=6671</p\n></html\n>'

In [74]:
# Upload new models into the database
bg_loader.upload_from_file(filename='../test_models/IEEE13.xml')
bg_loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
bg_loader.upload_from_file(filename='../test_output/breaker_half.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="21"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="29"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="501" milliseconds="50"/>

To verify that all feeder data can be obtained, we create a new CIMantic Graphs model and then query for the classes Substation and Feeder. We can also obtain the total amount of aggregate load from both feeders that are served by the substation.

In [75]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [76]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "ed5fc824-37f7-4cd0-9308-0f5a1aecbcf6": {
        "mRID": "ed5fc824-37f7-4cd0-9308-0f5a1aecbcf6",
        "name": "breaker_half_sub",
        "ConnectivityNodes": "['89a75774-481f-4c45-a207-39f2f37d1086', '9bc0639a-9c02-442a-9021-feb0ab263a27', 'a5101289-6677-4591-b3f2-2b6bc07f3b81', 'd7a3151b-f079-42f8-af7a-1d5f1ae91d9e', 'd88d3c43-53df-41a7-9ff2-9ae598bd1a58', '02b9f9b5-be34-46ef-8df0-423324815c73', '046e21f8-b976-4714-87b6-fdfe879c8c6e', '04792457-e2cc-465c-8e74-679d2d9a0bf4', '1e26ca4e-e460-4c52-a3ff-ec3442221b0c', '1f5b498c-1002-4d16-befc-c8807a6ec3f6', '2ca5d713-ab28-444b-8c1c-a64bfb4995ea', '2cc57af6-5c71-40e1-95dc-813f1350a227', '3ed78b52-fb85-494c-b788-4b93f339e489', '427e7786-a37d-4e98-8546-6f6a9d0abe4d', '4a55f898-c1d0-4d9a-b764-4f570a865297', '68144d36-739d-43b4-b0f2-54dad9cf34fb', '6cd333fa-ff81-4382-a13a-fbc61a4b8e0e', '729fac0c-06f8-4352-b10b-959926639255']",
        "Equipments": "['8456bc50-3b88-44de-b4cb-03913a0ae18d', '8533519c-193b-4c54-941a-753666f1cf1b', '88

In [77]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', '94F822E0-7130-4205-8597-B47110BBEF4B', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', '8C58660F-C62C-4903-BE72-22F1255B1E62', '76D6D03C-96

In [78]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


As part of the round-trip test, it is also possible to merge the substation and both feeders into a single XML file 

In [79]:
utils.get_all_data(network)
utils.write_xml(network, '../test_output/breaker_half_and_feeders.xml')